In [1]:
path = '/home/naman21266/Atul_ECG_Codes/Atul codes/codes/data/'

In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, activations, regularizers, optimizers, losses, metrics, callbacks
import matplotlib.pyplot as plt
from tensorflow.keras import backend as K
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import wfdb
import scipy.signal as signal
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

2024-09-29 08:16:33.280223: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-29 08:16:33.293350: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-29 08:16:33.297315: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-29 08:16:33.306896: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-09-29 08:16:33.965071: W tensorflow/compiler/tf2

In [3]:
x_train = np.load(path + 'x_train.npy')
y_train = np.load(path + 'y_train.npy')
x_test  = np.load(path + 'x_test.npy')
y_test  = np.load(path + 'y_test.npy')

# x_train = x_train.transpose(0, 2, 1)                # transpose working correctly
# x_test  = x_test.transpose(0, 2, 1)

# x_train = x_train.reshape(19267, 12, 1000, 1)       # Added another channel
# x_test  = x_test.reshape(2163, 12, 1000, 1)

print("X_train :", x_train.shape)
print("Y_train :", y_train.shape)
print("X_test  :", x_test.shape)
print("Y_test  :", y_test.shape)
print('Data loaded')

X_train : (19267, 1000, 12)
Y_train : (19267, 5)
X_test  : (2163, 1000, 12)
Y_test  : (2163, 5)
Data loaded


In [4]:
import pandas as pd

label_cnt = {}
for i in y_train:
    key = tuple(i.tolist())
    if key not in label_cnt:
        label_cnt[key] = 1
    else:
        label_cnt[key] += 1

df = pd.DataFrame(list(label_cnt.items()), columns=['Label Combination', 'Count'])
df = df.sort_values(by='Count', ascending=False)
print(df)

   Label Combination  Count
0    (0, 0, 0, 1, 0)   8170
1    (0, 0, 1, 0, 0)   2282
2    (0, 0, 0, 0, 1)   2163
4    (1, 0, 0, 0, 0)   1525
7    (1, 0, 1, 0, 0)   1167
10   (0, 1, 0, 0, 1)    710
5    (0, 0, 1, 0, 1)    541
3    (0, 1, 0, 0, 0)    480
8    (1, 0, 0, 0, 1)    434
13   (1, 0, 0, 1, 0)    362
11   (0, 1, 1, 0, 1)    320
6    (1, 1, 0, 0, 0)    273
12   (1, 0, 1, 0, 1)    202
15   (1, 1, 0, 0, 1)    186
9    (0, 1, 1, 0, 0)    166
14   (1, 1, 1, 0, 1)    142
17   (1, 1, 1, 0, 0)    112
16   (0, 0, 0, 1, 1)     24
18   (1, 0, 0, 1, 1)      5
19   (0, 1, 0, 1, 0)      2
20   (1, 1, 1, 1, 0)      1


In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

class SelfAttention1D(nn.Module):
    def __init__(self, channels, size):
        super(SelfAttention1D, self).__init__()
        self.channels = channels
        self.size = size
        self.mha = nn.MultiheadAttention(channels, 4, batch_first=True)
        self.ln = nn.LayerNorm([channels])
        self.ff_self = nn.Sequential(
            nn.LayerNorm([channels]),
            nn.Linear(channels, channels),
            nn.GELU(),
            nn.Linear(channels, channels),
        )

    def forward(self, x):
        x = x.view(-1, self.size, self.channels)
        x_ln = self.ln(x)
        attention_value, _ = self.mha(x_ln, x_ln, x_ln)
        attention_value = attention_value + x
        attention_value = self.ff_self(attention_value) + attention_value
        return attention_value.view(-1, self.channels, self.size)


class DoubleConv1D(nn.Module):
    def __init__(self, in_channels, out_channels, mid_channels=None, residual=False):
        super().__init__()
        self.residual = residual
        if not mid_channels:
            mid_channels = out_channels
        self.double_conv = nn.Sequential(
            nn.Conv1d(in_channels, mid_channels, kernel_size=3, padding=1, bias=False),
            nn.GroupNorm(1, mid_channels),
            nn.GELU(),
            nn.Conv1d(mid_channels, out_channels, kernel_size=3, padding=1, bias=False),
            nn.GroupNorm(1, out_channels),
        )

    def forward(self, x):
        if self.residual:
            return F.gelu(x + self.double_conv(x))
        else:
            return self.double_conv(x)


class Down1D(nn.Module):
    def __init__(self, in_channels, out_channels, emb_dim=1000):
        super().__init__()
        self.maxpool_conv = nn.Sequential(
            nn.MaxPool1d(2),
            DoubleConv1D(in_channels, in_channels, residual=True),
            DoubleConv1D(in_channels, out_channels),
        )

        self.emb_layer = nn.Sequential(
            nn.SiLU(),
            nn.Linear(
                emb_dim,
                out_channels
            ),
        )

    def forward(self, x, t):
        x = self.maxpool_conv(x)
        emb = self.emb_layer(t)[:, :, None].repeat(1, 1, x.shape[-1])
        return x + emb


class Up1D(nn.Module):
    def __init__(self, in_channels, out_channels, emb_dim=1000):
        super().__init__()

        self.up = nn.Upsample(scale_factor=2, mode="linear", align_corners=True)
        self.conv = nn.Sequential(
            DoubleConv1D(in_channels, in_channels, residual=True),
            DoubleConv1D(in_channels, out_channels, in_channels // 2),
        )

        self.emb_layer = nn.Sequential(
            nn.SiLU(),
            nn.Linear(
                emb_dim,
                out_channels
            ),
        )

    def forward(self, x, skip_x, t):
        x = self.up(x)
        x = torch.cat([skip_x, x], dim=1)
        x = self.conv(x)
        emb = self.emb_layer(t)[:, :, None].repeat(1, 1, x.shape[-1])
        return x + emb


class UNet1D(nn.Module):
    def __init__(self, c_in=12, c_out=12, time_dim=1000, device="cuda"):
        super().__init__()
        self.device = device
        self.time_dim = time_dim
        
        self.inc = DoubleConv1D(c_in, 64)          # Input: (N, c_in=12, L=1000) -> Output: (N, 64, 1000)
        self.down1 = Down1D(64, 128)               # Input: (N, 64, 1000) -> Output: (N, 128, 500)
        self.sa1 = SelfAttention1D(128, 500)       # Input: (N, 128, 500) -> Output: (N, 128, 500)
        
        self.down2 = Down1D(128, 256)              # Input: (N, 128, 500) -> Output: (N, 256, 250)
        self.sa2 = SelfAttention1D(256, 250)       # Input: (N, 256, 250) -> Output: (N, 256, 250)
        
        self.down3 = Down1D(256, 256)              # Input: (N, 256, 250) -> Output: (N, 256, 125)
        self.sa3 = SelfAttention1D(256, 125)       # Input: (N, 256, 125) -> Output: (N, 256, 125)

        self.bot1 = DoubleConv1D(256, 512)         # Input: (N, 256, 125) -> Output: (N, 512, 125)
        self.bot2 = DoubleConv1D(512, 512)         # Input: (N, 512, 125) -> Output: (N, 512, 125)
        self.bot3 = DoubleConv1D(512, 256)         # Input: (N, 512, 125) -> Output: (N, 256, 125)

        self.up1 = Up1D(512, 128)                   # Input: (N, 256, 125) + (N, 512, 125) -> Output: (N, 128, 250)
        self.sa4 = SelfAttention1D(128, 250)       # Input: (N, 128, 250) -> Output: (N, 128, 250)
        
        self.up2 = Up1D(256, 64)                    # Input: (N, 128, 250) + (N, 256, 250) -> Output: (N, 64, 500)
        self.sa5 = SelfAttention1D(64, 500)        # Input: (N, 64, 500) -> Output: (N, 64, 500)
        
        self.up3 = Up1D(128, 64)                    # Input: (N, 64, 500) + (N, 64, 500) -> Output: (N, 64, 1000)
        self.sa6 = SelfAttention1D(64, 1000)       # Input: (N, 64, 1000) -> Output: (N, 64, 1000)
        
        self.outc = nn.Conv1d(64, c_out, kernel_size=1)  # Input: (N, 64, 1000) -> Output: (N, c_out=12, 1000)

    def pos_encoding(self, t, channels):
        inv_freq = 1.0 / (
            10000
            ** (torch.arange(0, channels, 2, device=self.device).float() / channels)
        )
        pos_enc_a = torch.sin(t.repeat(1, channels // 2) * inv_freq)
        pos_enc_b = torch.cos(t.repeat(1, channels // 2) * inv_freq)
        pos_enc = torch.cat([pos_enc_a, pos_enc_b], dim=-1)
        return pos_enc

    def forward(self, x, t):
        t = t.unsqueeze(-1).type(torch.float)  # Input: (N, L=1000) -> Output: (N, 1)
        t = self.pos_encoding(t, self.time_dim) # Output: (N, time_dim=1000)

        x1 = self.inc(x)                         # Input: (N, c_in=12, L=1000) -> Output: (N, 64, 1000)
        x2 = self.down1(x1, t)                  # Input: (N, 64, 1000) -> Output: (N, 128, 500)
        x2 = self.sa1(x2)                       # Input: (N, 128, 500) -> Output: (N, 128, 500)
        
        x3 = self.down2(x2, t)                  # Input: (N, 128, 500) -> Output: (N, 256, 250)
        x3 = self.sa2(x3)                       # Input: (N, 256, 250) -> Output: (N, 256, 250)

        x4 = self.down3(x3, t)                  # Input: (N, 256, 250) -> Output: (N, 256, 125)
        x4 = self.sa3(x4)                       # Input: (N, 256, 125) -> Output: (N, 256, 125)

        x4 = self.bot1(x4)                      # Input: (N, 256, 125) -> Output: (N, 512, 125)
        x4 = self.bot2(x4)                      # Input: (N, 512, 125) -> Output: (N, 512, 125)
        x4 = self.bot3(x4)                      # Input: (N, 512, 125) -> Output: (N, 256, 125)

        x = self.up1(x4, x3, t)                  # Input: (N, 256, 125) + (N, 512, 125) -> Output: (N, 128, 250)
        x = self.sa4(x)                          # Input: (N, 128, 250) -> Output: (N, 128, 250)

        x = self.up2(x, x2, t)                   # Input: (N, 128, 250) + (N, 256, 250) -> Output: (N, 64, 500)
        x = self.sa5(x)                          # Input: (N, 64, 500) -> Output: (N, 64, 500)

        x = self.up3(x, x1, t)                   # Input: (N, 64, 500) + (N, 64, 500) -> Output: (N, 64, 1000)
        x = self.sa6(x)                          # Input: (N, 64, 1000) -> Output: (N, 64, 1000)

        output = self.outc(x)                    # Input: (N, 64, 1000) -> Output: (N, c_out=12, 1000)

        return output

In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import math

# Assuming the UNet1D model is already defined as per your previous code

class GaussianDiffusion(nn.Module):
    def __init__(self, model, timesteps=1000, beta_start=1e-4, beta_end=0.02, device="cuda"):
        super(GaussianDiffusion, self).__init__()
        self.model = model
        self.timesteps = timesteps
        self.device = device
        
        # Linearly spaced betas for noise schedule
        self.betas = torch.linspace(beta_start, beta_end, timesteps).to(self.device)
        self.alphas = 1.0 - self.betas
        self.alphas_cumprod = torch.cumprod(self.alphas, dim=0)
        self.alphas_cumprod_prev = torch.cat([torch.tensor([1.0], device=self.device), self.alphas_cumprod[:-1]])

        # Calculating the standard deviation for the forward process noise
        self.sqrt_alphas_cumprod = torch.sqrt(self.alphas_cumprod)
        self.sqrt_one_minus_alphas_cumprod = torch.sqrt(1.0 - self.alphas_cumprod)

    def q_sample(self, x_0, t, noise=None):
        """
        Forward process: Sample from q(x_t | x_0) by adding Gaussian noise to the data.
        """
        if noise is None:
            noise = torch.randn_like(x_0)
        return (
            self.sqrt_alphas_cumprod[t, None, None] * x_0 +
            self.sqrt_one_minus_alphas_cumprod[t, None, None] * noise
        )

    def p_losses(self, x_0, t, noise=None):
        """
        Training objective: Predict the noise that was added at timestep `t`.
        """
        if noise is None:
            noise = torch.randn_like(x_0)

        x_t = self.q_sample(x_0, t, noise)
        predicted_noise = self.model(x_t, t)

        return F.mse_loss(predicted_noise, noise)

    @torch.no_grad()
    def p_sample(self, x, t, t_index):
        """
        Reverse process: Sample from p(x_{t-1} | x_t)
        """
        betas_t = self.betas[t].reshape(-1, 1, 1)
        sqrt_one_minus_alphas_cumprod_t = self.sqrt_one_minus_alphas_cumprod[t].reshape(-1, 1, 1)
        sqrt_recip_alphas_t = torch.sqrt(1.0 / self.alphas[t]).reshape(-1, 1, 1)

        # Predict noise
        predicted_noise = self.model(x, t)

        # Compute the mean of p(x_{t-1} | x_t)
        model_mean = sqrt_recip_alphas_t * (x - betas_t * predicted_noise / sqrt_one_minus_alphas_cumprod_t)

        if t_index == 0:
            return model_mean
        else:
            noise = torch.randn_like(x)
            return model_mean + torch.sqrt(self.betas[t]).reshape(-1, 1, 1) * noise

    @torch.no_grad()
    def p_sample_loop(self, shape):
        """
        Reverse process: Iterate over the timesteps to generate a sample from pure noise.
        """
        b = shape[0]
        img = torch.randn(shape, device=self.device)
        imgs = []

        for i in reversed(range(0, self.timesteps)):
            img = self.p_sample(img, torch.full((b,), i, device=self.device, dtype=torch.long), i)
            imgs.append(img.cpu().numpy())

        return imgs

    def sample(self, image_shape):
        return self.p_sample_loop(image_shape)

# Assuming UNet1D is already implemented from the previous message.
# Initializing the diffusion model

device = "cuda" if torch.cuda.is_available() else "cpu"
unet_model = UNet1D(c_in=12, c_out=12, time_dim=1000, device=device).to(device)

diffusion = GaussianDiffusion(unet_model, timesteps=1000, device=device)

In [7]:
import torch.optim as optim

# Assuming you have the data in variables `train_data` (shape: (N, 12, 1000)) and `train_labels` (optional)
training_data = x_train
x_train_transposed = np.transpose(training_data, (0, 2, 1))
x_train_tensor = torch.from_numpy(x_train_transposed).float()  # Convert to float if needed

# Dataset and DataLoader
train_dataset = torch.utils.data.TensorDataset(x_train_tensor)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)

# Optimizer
optimizer = optim.Adam(diffusion.model.parameters(), lr=2e-4)

def train_diffusion_model(diffusion, train_loader, epochs=10):
    diffusion.train()
    
    for epoch in range(epochs):
        for batch_idx, (x,) in enumerate(train_loader):
            x = x.to(device)
            optimizer.zero_grad()

            # Random timestep t for each batch element
            t = torch.randint(0, diffusion.timesteps, (x.shape[0],), device=device).long()

            # Compute loss
            loss = diffusion.p_losses(x, t)

            # Backpropagation
            loss.backward()
            optimizer.step()

            if batch_idx % 100 == 0:
                print(f"Epoch [{epoch+1}/{epochs}] Batch [{batch_idx}/{len(train_loader)}] Loss: {loss.item():.4f}")

# Train the model
train_diffusion_model(diffusion, train_loader, epochs=20)


Epoch [1/20] Batch [0/603] Loss: 1.5007
Epoch [1/20] Batch [100/603] Loss: 0.2090
Epoch [1/20] Batch [200/603] Loss: 0.1171
Epoch [1/20] Batch [300/603] Loss: 0.1143
Epoch [1/20] Batch [400/603] Loss: 0.1185
Epoch [1/20] Batch [500/603] Loss: 0.0599
Epoch [1/20] Batch [600/603] Loss: 0.0479
Epoch [2/20] Batch [0/603] Loss: 0.0779
Epoch [2/20] Batch [100/603] Loss: 0.0541
Epoch [2/20] Batch [200/603] Loss: 0.0419
Epoch [2/20] Batch [300/603] Loss: 0.0478
Epoch [2/20] Batch [400/603] Loss: 0.0335
Epoch [2/20] Batch [500/603] Loss: 0.0441
Epoch [2/20] Batch [600/603] Loss: 0.0531
Epoch [3/20] Batch [0/603] Loss: 0.0311
Epoch [3/20] Batch [100/603] Loss: 0.0339
Epoch [3/20] Batch [200/603] Loss: 0.0243
Epoch [3/20] Batch [300/603] Loss: 0.0263
Epoch [3/20] Batch [400/603] Loss: 0.0282
Epoch [3/20] Batch [500/603] Loss: 0.0425
Epoch [3/20] Batch [600/603] Loss: 0.0248
Epoch [4/20] Batch [0/603] Loss: 0.0268
Epoch [4/20] Batch [100/603] Loss: 0.0307
Epoch [4/20] Batch [200/603] Loss: 0.0265
